A context manager is responsible for a resource within a code block, possibly creating it when the block is entered and then cleaning it up after the block is exited. For example, files support the context manager API to make it easy to ensure they are closed after all reading or writing is done.

A context manager is enabled by the with statement, and the API involves two methods. The \__enter\__() method is run when execution flow enters the code block inside the with. It returns an object to be used within the context. When execution flow leaves the with block, the \__exit\__() method of the context manager is called to clean up any resources being used.



In [3]:
class C(object):
    
    value = 10
    
    def __init__(self):
        print 'Initialization'
        
    def __enter__(self):
        print 'Entering'
        #self is now assigned to f
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print 'Exiting'
        
        
with C() as f:
    print f.value
    print 'Doing something!'

Initialization
Entering
10
Doing something!
Exiting


The \__exit\__() method receives arguments containing details of any exception raised in the with block.

In [6]:
class C(object):
    
    def __init__(self, handle_error):
        print 'Initialization'
        self.handle_error = handle_error
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__()')
        print('  exc_type =', exc_type)
        print('  exc_val  =', exc_val)
        print('  exc_tb   =', exc_tb)
        return self.handle_error
    
with C(True):
    raise RuntimeError('error message handled')

Initialization
__exit__()
('  exc_type =', <type 'exceptions.RuntimeError'>)
('  exc_val  =', RuntimeError('error message handled',))
('  exc_tb   =', <traceback object at 0x106277cf8>)


In [8]:
with C(False):
    raise RuntimeError('error message propagated')

Initialization
__exit__()
('  exc_type =', <type 'exceptions.RuntimeError'>)
('  exc_val  =', RuntimeError('error message propagated',))
('  exc_tb   =', <traceback object at 0x106277c68>)


RuntimeError: error message propagated

If the context manager can handle the exception, __exit__() should return a true value to indicate that the exception does not need to be propagated. Returning false causes the exception to be re-raised after \__exit\__() returns.


## Context Manager as Function Decorators
The class ContextDecorator adds support to regular context manager classes to let them be used as function decorators as well as context managers.



In [17]:
import contextlib2

class Context(contextlib2.ContextDecorator):
    
    def __init__(self, how_used):
        self.how_used = how_used
        print('__init__({})'.format(how_used))
        
    def __enter__(self):
        print('__enter__({})'.format(self.how_used))
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__({})'.format(self.how_used))


@Context('as decorator')
def func(message):
    print message
    

print
with Context('as context manager'):
    print 'Doing work in the context'
    
print
func('Doing work in the wrapped function')

__init__(as decorator)

__init__(as context manager)
__enter__(as context manager)
Doing work in the context
__exit__(as context manager)

__enter__(as decorator)
Doing work in the wrapped function
__exit__(as decorator)


One difference with using the context manager as a decorator is that the value returned by \__enter\__() is not available inside the function being decorated, unlike when using with and as. Arguments passed to the decorated function are available in the usual way.

## From Generator to Context Manager
Creating context managers the traditional way, by writing a class with \__enter\__() and \__exit\__() methods, is not difficult. But sometimes writing everything out fully is extra overhead for a trivial bit of context. In those sorts of situations, use the contextmanager() decorator to convert a generator function into a context manager.

In [21]:
import contextlib

@contextlib.contextmanager
def make_context():
    print '  entering'
    try:
        yield 'inside value'
    except RuntimeError as err:
        print '  ERROR:', err
    finally:
        print '  exiting'
        
print('Normal:')
with make_context() as value:
    print('  inside with statement:', value)
    
print('\nHandled error:')
with make_context() as value:
    raise RuntimeError('showing example of handling an error')

print('\nUnhandled error:')
with make_context() as value:
    raise ValueError('this exception is not handled')


Normal:
  entering
('  inside with statement:', 'inside value')
  exiting

Handled error:
  entering
  ERROR: showing example of handling an error
  exiting

Unhandled error:
  entering
  exiting


ValueError: this exception is not handled

The generator should initialize the context, yield exactly one time, then clean up the context. The value yielded, if any, is bound to the variable in the as clause of the with statement. Exceptions from within the with block are re-raised inside the generator, so they can be handled there.

## Closing Open Handles

The file class supports the context manager API directly, but some other objects that represent open handles do not. The example given in the standard library documentation for contextlib is the object returned from urllib.urlopen(). There are other legacy classes that use a close() method but do not support the context manager API. To ensure that a handle is closed, use closing() to create a context manager for it.



In [24]:
import contextlib

class Door(object):
    
    value = 2
    
    def __init__(self):
        print 'Initialization'
        self.status = 'open'
        
    def close(self):
        print 'Closing'
        self.status = 'closed'
        
print 'Normal Example:'
with contextlib.closing(Door()) as door:
    print door.status
print door.status

print('\nError handling example:')
try:
    with contextlib.closing(Door()) as door:
        print('  raising from inside with statement')
        raise RuntimeError('error message')
except Exception as err:
    print('  Had an error:', err)

Normal Example:
Initialization
open
Closing
closed

Error handling example:
Initialization
  raising from inside with statement
Closing
('  Had an error:', RuntimeError('error message',))


## Redirecting output streams
Poorly designed library code may write directly to sys.stdout or sys.stderr, without providing arguments to configure different output destinations. The redirect_stdout() and redirect_stderr() context managers can be used to capture output from functions like this, for which the source cannot be changed to accept a new output argument.

In [31]:
from contextlib2 import redirect_stdout, redirect_stderr
import io
import sys

def misbehaving_function(a):
    sys.stdout.write('(stdout) A: {}\n'.format(a))
    sys.stderr.write('(stderr) A: {}\n'.format(a))

capture = io.BytesIO()
with redirect_stdout(capture), redirect_stderr(capture):
    misbehaving_function(5)
    
print capture.getvalue()

(stdout) A: 5
(stderr) A: 5



## Dynamic Context Manager Stacks
Most context managers operate on one object at a time, such as a single file or database handle. In these cases, the object is known in advance and the code using the context manager can be built around that one object. In other cases, a program may need to create an unknown number of objects in a context, while wanting all of them to be cleaned up when control flow exits the context. ExitStack was created to handle these more dynamic cases.

An ExitStack instance maintains a stack data structure of cleanup callbacks. The callbacks are populated explicitly within the context, and any registered callbacks are called in the reverse order when control flow exits the context. The result is like having multple nested with statements, except they are established dynamically.

In [33]:
import contextlib

@contextlib.contextmanager
def make_context(i):
    print '{} entering'.format(i)
    yield {}
    print '{} exiting'.format(i)
    
def variable_stack(n, msg):
    with contextlib2.ExitStack() as stack:
        for i in range(n):
            stack.enter_context(make_context(i))
        print(msg)

variable_stack(2, 'inside context')

0 entering
1 entering
inside context
1 exiting
0 exiting
